In [ ]:
# !pip install -qU langchain_ollama

In [2]:
#Imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [3]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Created a chunk of size 1088, which is longer than the specified 1000


Total number of chunks: 123
Document types found: {'products', 'contracts', 'company', 'employees'}


In [4]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite
DB_NAME = "vector_db"

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Delete if already exists

if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


In [5]:
#run a quick test - should return a list of documents = 4
question = "contract with Belvedere Insurance"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [6]:
docs[0]

Document(id='e9bf36ed-7831-4b6c-a22e-7f5e12353016', metadata={'source': 'knowledge-base\\contracts\\Contract with Belvedere Insurance for Markellm.md', 'doc_type': 'contracts'}, page_content='# Contract with Belvedere Insurance for Markellm\n\n## Terms\nThis Contract ("Agreement") is made and entered into as of [Date] by and between Insurellm, Inc., a corporation registered in the United States, ("Provider") and Belvedere Insurance, ("Client"). \n\n1. **Service Commencement**: The services described herein will commence on [Start Date].\n2. **Contract Duration**: This Agreement shall remain in effect for a period of 1 year from the Commencement Date, unless terminated earlier in accordance with the termination clause of this Agreement.\n3. **Fees**: Client agrees to pay a Basic Listing Fee of $199/month for accessing the Markellm platform along with a performance-based pricing of $25 per lead generated.\n4. **Payment Terms**: Payments shall be made monthly, in advance, with invoices is

In [7]:
# create a new Chat with Ollama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
MODEL = "llama3.2:latest"
llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23816\1192425351.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [8]:
# Let's try a simple question

query = "Explain contract with Belvedere Insurance"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

The provided text is a sample contract between Insurellm, Inc. (the provider) and Belvedere Insurance (the client). The contract outlines the terms and conditions of their partnership, which involves using the Markellm platform for customer acquisition.

Here's a breakdown of the key points:

**Contract Overview**

* The contract commences on a specific start date and remains in effect for 1 year, unless terminated earlier.
* Belvedere Insurance agrees to pay a monthly Basic Listing Fee of $199 and a performance-based pricing of $25 per lead generated.

**Payment Terms**

* Payments are due monthly, in advance, with invoices issued on the 1st of each month. Clients must pay within 15 days of receiving the invoice.

**Support and Training**

* Technical support is available from 9 AM to 7 PM EST, Monday through Friday via email and phone.
* Insurellm agrees to respond to all support queries within 24 business hours, with emergency support prioritized.
* A comprehensive training session 

In [9]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the  LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [12]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [30]:
# Store the rating
user_rating = []

In [36]:
def store_rating(value): # we store the rating from the user
    global user_rating
    user_rating.append(value)
    return f"Thanks! Your rating ({value}/5) has been recorded."

In [38]:
with gr.Blocks() as demo:
    gr.Markdown("### Chat with the assistant and then rate your experience")

    # ChatInterface
    chatbot = gr.ChatInterface(fn=chat, type="messages")

    # Rating section
    gr.Markdown("#### Rate the assistant (1-5)")
    rating_slider = gr.Slider(minimum=1, maximum=5, step=1, label="Your Rating")
    submit_button = gr.Button("Submit Rating")
    output_text = gr.Textbox(label="Feedback Confirmation")

    # Link rating button to function
    submit_button.click(fn=store_rating, inputs=rating_slider, outputs=output_text)

demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [40]:
user_rating

[2, 3, 4]